 ## Creating a Snowflake UDF from almost any johnsnowlabs model

Prerequisites:
- [Docker](https://www.docker.com/) installed  
- [snowflake-connector-python](https://pypi.org/project/snowflake-connector-python/) library installed, i.e run `pip install snowflake-connector-python` and access to Snowflake user, password, account-url and acces to `ACCOUNTADMIN` role
- [johnsnowlabs](https://pypi.org/project/johnsnowlabs) library installed i.e `pip install johnsnowlabs`



Common issues:
```
 snowflake.connector.errors.ProgrammingError: 000606 (57P03): No active warehouse selected in the current session.  Select an active warehouse with the 'use warehouse' command.
```
1. Add a default warehouse in user settings via Snowflake UI
2. Add default role ACOCUNTADMIN in user settings via Snowflake UI


In [1]:
from tests.utilsz import secrets
import os
# fill this in with your credentials
os.environ['SNOWFLAKE_USER'] = 'MY_SNOWFLAKE_USER'
os.environ['SNOWFLAKE_PASSWORD'] = 'MY_SNOWFLAKE_PASSWORD'
os.environ['SNOWFLAKE_ACCOUNT'] = 'MY_SNOWFLAKE_ACCOUNT' # i.e abc123.us-west-2

from tests.utilsz import secrets
import os
os.environ['SNOWFLAKE_USER'] = secrets.snowflake_user
os.environ['SNOWFLAKE_PASSWORD'] = secrets.snowflake_password
os.environ['SNOWFLAKE_ACCOUNT'] = secrets.snowflake_account # 'ekb84445.us-west-2' 


In [2]:
from johnsnowlabs import nlp

# Create common setup
# https://docs.snowflake.com/en/developer-guide/snowpark-container-services/tutorials/common-setup#introduction 
role_name, db_name, warehouse_name, schema_name, compute_pool_name, repo_url = nlp.snowflake_common_setup(
    snowflake_user=os.environ['SNOWFLAKE_USER'],
    snowflake_account=os.environ['SNOWFLAKE_ACCOUNT'],
    snowflake_password=os.environ['SNOWFLAKE_PASSWORD'],
)

# Create a container, test it locally, login to repo and push
model_to_deploy = 'tokenize'
udf_name = nlp.deploy_as_snowflake_udf(
    model_to_deploy,
    repo_url=repo_url,
    role_name=role_name,
    database_name=db_name,
    warehouse_name=warehouse_name,
    schema_name=schema_name,
    compute_pool_name=compute_pool_name,
    snowflake_user=os.environ['SNOWFLAKE_USER'],
    snowflake_account=os.environ['SNOWFLAKE_ACCOUNT'],
    snowflake_password=os.environ['SNOWFLAKE_PASSWORD'],
)


Created Role test_role and access granted to christian2
Created Database tutorial_db
Created Warehouse tutorial_warehouse
Created Compute Pool tutorial_compute_pool
Created Schema data_schema
Created Repository tutorial_repository
Created Stage tutorial_stage
[(datetime.datetime(2024, 1, 23, 14, 31, 0, 500000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), 'TUTORIAL_REPOSITORY', 'TUTORIAL_DB', 'DATA_SCHEMA', 'a0524544206961-vzb99979.registry.snowflakecomputing.com/tutorial_db/data_schema/tutorial_repository', 'TEST_ROLE', 'ROLE', '')]
Created Snowflake Container Repository a0524544206961-vzb99979.registry.snowflakecomputing.com/tutorial_db/data_schema/tutorial_repository
👷 Executing docker image rm -f tokenize-img
✅ Success running ['docker image rm -f tokenize-img']
Image 'tokenize-img' destroyed.
📋 Loading license number 0 from /home/ckl/.johnsnowlabs/licenses/license_number_{number}_for_Spark-Healthcare_Spark-OCR.json
👷 Executing cd /home/ckl/Documents/freelance/

## Lets query the UDF via Snowflake client.
The next two cells  can be run inside a Snowflake based Notebook or Python script or locally

In [3]:
import snowflake.connector
conn = snowflake.connector.connect(
    user=secrets.snowflake_user,
    password=secrets.snowflake_password,
    account=secrets.snowflake_account,
    warehouse=warehouse_name,
    database=db_name,
    schema=schema_name,
    role=role_name,
)

In [4]:
import json

data = 'hello World'
cmd_query_udf = f"""SELECT {udf_name}('{data}')"""
cur = conn.cursor()
cur.execute(cmd_query_udf.format(data=data))
for row in cur:
    data = json.loads(row[0])
    print(data)
cur.close()

{'document': 'hello World', 'index': 0, 'origin_index': 0, 'sentence': ['hello World'], 'token': ['hello', 'World']}


True

## Lets create a licensed model endpoint


In [5]:
from tests.utilsz import secrets
from johnsnowlabs import nlp

# Create common setup
role_name, db_name, warehouse_name, schema_name, compute_pool_name, repo_url = nlp.snowflake_common_setup(
    secrets.snowflake_user, secrets.snowflake_account, secrets.snowflake_password,
)

license = '/home/ckl/Downloads/license.json'
# Create a container, test it locally, login to repo and push
model = 'en.explain_doc.clinical_granular'  
udf_name = nlp.deploy_as_snowflake_udf(model,
                                       repo_url=repo_url,
                                       role_name=role_name,
                                       database_name=db_name,
                                       warehouse_name=warehouse_name,
                                       schema_name=schema_name,
                                       compute_pool_name=compute_pool_name,
                                       snowflake_user=os.environ['SNOWFLAKE_USER'],
                                       snowflake_account=os.environ['SNOWFLAKE_ACCOUNT'],
                                       snowflake_password=os.environ['SNOWFLAKE_PASSWORD'],
                                       license_path=license,
                                       )


Created Role test_role and access granted to christian2
Created Database tutorial_db
Created Warehouse tutorial_warehouse
Created Compute Pool tutorial_compute_pool
Created Schema data_schema
Created Repository tutorial_repository
Created Stage tutorial_stage
[(datetime.datetime(2024, 1, 23, 14, 31, 0, 500000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), 'TUTORIAL_REPOSITORY', 'TUTORIAL_DB', 'DATA_SCHEMA', 'a0524544206961-vzb99979.registry.snowflakecomputing.com/tutorial_db/data_schema/tutorial_repository', 'TEST_ROLE', 'ROLE', '')]
Created Snowflake Container Repository a0524544206961-vzb99979.registry.snowflakecomputing.com/tutorial_db/data_schema/tutorial_repository
👷 Executing docker image rm -f en-explain-doc-clinical-granular-img
✅ Success running ['docker image rm -f en-explain-doc-clinical-granular-img']
Image 'en-explain-doc-clinical-granular-img' destroyed.
👌 License info detected in file /home/ckl/Downloads/license.json
👷 Executing cd /home/ckl/Document

In [6]:
import snowflake.connector

conn = snowflake.connector.connect(
    user=secrets.snowflake_user,
    password=secrets.snowflake_password,
    account=secrets.snowflake_account,
    warehouse=warehouse_name,
    database=db_name,
    schema=schema_name,
    role=role_name,
)


In [7]:
import json
data = """
The patient admitted for gastrointestinal pathology, under working treatment.
History of prior heart murmur with echocardiogram findings as above on March 1998.
Echocardiogram from today indicates left ventricular function is normal but left atrial enlargement.
Based on the above findings, we will treat her medically with ACE inhibitors 10 mg, p.o, daily. Also we will give Furosemide 40 mg, p.o later and see how she fares."""


udf_name = 'en_explain_doc_clinical_granular_udf'
cmd_query_udf = f"""SELECT {udf_name}('{data}')"""
cur = conn.cursor()
cur.execute(cmd_query_udf.format(data=data))
for row in cur:
    data = json.loads(row[0])
    print(data)
cur.close()

{'all_relations_result': 'is_finding_of', 'assertion': ['Past', 'Past', 'Present', 'Present', 'Present', 'Present', 'Planned', 'Planned'], 'assertion_confidence': ['1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0'], 'assertion_origin_chunk': ['Heart_Disease', 'Test', 'Test', 'Test', 'Test_Result', 'Heart_Disease', 'Drug_Ingredient', 'Drug_Ingredient'], 'assertion_origin_sentence': ['1', '1', '2', '2', '2', '2', '3', '4'], 'document': '\nThe patient admitted for gastrointestinal pathology, under working treatment.\nHistory of prior heart murmur with echocardiogram findings as above on March 1998.\nEchocardiogram from today indicates left ventricular function is normal but left atrial enlargement.\nBased on the above findings, we will treat her medically with ACE inhibitors 10 mg, p.o, daily. Also we will give Furosemide 40 mg, p.o later and see how she fares.', 'entities_assertion_ner_chunk': ['heart murmur', 'echocardiogram', 'Echocardiogram', 'left ventricular function', 'normal'

True